In [1]:
# -*- coding: utf-8 -*-
"""
Created on Mon Jan 29 20:48:23 2018

@author: jsc5565
"""


import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import time
tf.reset_default_graph()

with tf.device('/gpu:0'):
    mnist = input_data.read_data_sets("mnist", one_hot=True)

    learning_rate = 0.001
    train_epochs = 20
    train_size = 100
    display_step = 1
    filter_size = 20
    img_size = 14

    x = tf.placeholder(tf.float32, [None, 28*28])
    y = tf.placeholder(tf.float32, [None, 10])

    row_w = tf.get_variable(name="row_w", shape=[img_size, filter_size], initializer=tf.ones_initializer())
    col_w = tf.get_variable(name="col_w", shape=[img_size, filter_size], initializer=tf.ones_initializer())
    filter_w = tf.Variable(tf.random_normal([7,7,1,filter_size], stddev=0.01))
    #output = tf.get_variable(name="sum_row_w", shape=[img_size*img_size*filter_size, 10], initializer=tf.ones_initializer())
    output = tf.get_variable(name="sum_row_w", shape=[img_size*filter_size, 10], initializer=tf.ones_initializer())
    logit = tf.get_variable(name="logit", shape=[img_size*img_size, 10], initializer=tf.ones_initializer())

    #x_weight = tf.get_variable(name="x_weight", shape=[img_size*img_size*filter_size], initializer=tf.ones_initializer())

    x_re = tf.reshape(x, [-1, 28, 28, 1])
    x_conv = tf.nn.conv2d(x_re, filter_w, strides=[1,1,1,1], padding='SAME')
    x_pool = tf.nn.max_pool(x_conv, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME') #?, 14, 14, filter_size

    """first conv2d"""
    mul1 = x_pool * row_w
    transpose = tf.transpose(mul1, perm=[0, 2, 1, 3])
    mul2 = transpose * col_w

    col_sum = tf.reduce_sum(mul2, axis=1)
    row_sum = tf.reduce_sum(mul2, axis=2)

    col_sum_re = tf.reshape(col_sum, [-1, img_size*filter_size])
    row_sum_re = tf.reshape(row_sum, [-1, img_size*filter_size])

    total_sum = col_sum_re + row_sum_re
    relu_add = tf.nn.relu(total_sum)

    #col_relu = tf.nn.relu(col_sum_re)
    #row_relu = tf.nn.relu(row_sum_re)

    #relu_add = tf.add(col_relu, row_relu)

    result = tf.matmul(relu_add, output)
    #x_reshape = tf.reshape(x_pool, [-1, img_size*img_size*filter_size])
    #x_reshape_mat = x_reshape * x_weight
    #result = tf.matmul(x_reshape_mat, output)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=result, labels=y))
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)



Extracting mnist/train-images-idx3-ubyte.gz
Extracting mnist/train-labels-idx1-ubyte.gz
Extracting mnist/t10k-images-idx3-ubyte.gz
Extracting mnist/t10k-labels-idx1-ubyte.gz
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [2]:
#gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.8)
config = tf.ConfigProto(log_device_placement=True)
config.gpu_options.allow_growth = True

with tf.Session(config=config) as sess:
    sess.run(tf.global_variables_initializer())
    start_time = time.time()
    for epoch in range(train_epochs):
        avg_cost = 0
        
        total_epochs = int(mnist.train.num_examples/ train_size)
        
        for i in range(total_epochs):
            batch_xs, batch_ys = mnist.train.next_batch(train_size)
            _,c, row= sess.run([optimizer, cost, row_w], {x: batch_xs, y: batch_ys})
            avg_cost += c/total_epochs
            
        if (epoch+1) % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", "{:9f}".format(avg_cost))
            
    prediction = tf.equal(tf.argmax(result, 1), tf.argmax(y, 1))            
            
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    print("Accuracy:", accuracy.eval({x: mnist.test.images, y:mnist.test.labels}))
    
    sess.close()
    
    end_time = time.time()
    
    print('time:', end_time-start_time)

InvalidArgumentError: Cannot assign a device for operation 'sum_row_w/Adam_1': Operation was explicitly assigned to /device:GPU:0 but available devices are [ /job:localhost/replica:0/task:0/device:CPU:0 ]. Make sure the device specification refers to a valid device.
	 [[Node: sum_row_w/Adam_1 = VariableV2[_class=["loc:@sum_row_w"], container="", dtype=DT_FLOAT, shape=[280,10], shared_name="", _device="/device:GPU:0"]()]]

Caused by op 'sum_row_w/Adam_1', defined at:
  File "/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/jsc5565/python3/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/jsc5565/python3/tensorflow/lib/python3.6/sit

In [ ]:
Accuracy: 0.984
time: 39.31117653846741